In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [20]:
df = pd.read_csv("data/cleaned/pubs.csv")

C:\Users\20211831\AppData\Local\Temp\ipykernel_13188\887412386.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/cleaned/pubs.csv")


### Filter for biomedical institutions

In [21]:
biomedical = ['amcpub', 'lumc', 'vumc', 'umcu'] 
subset = df[df['institution'].isin(biomedical)]
subset = subset.reset_index(drop=True)

### Datetime format for `year`

In [22]:
# datetime change, run once
for index, value in enumerate(subset['year']):
    subset.at[index, 'year'] = datetime.strptime(str(value), '%Y')
date_5_years_ago = datetime.now() - timedelta(days=5*365)
new_df = subset[subset['year'] >= date_5_years_ago]
new_df = new_df.reset_index(drop=True)

C:\Users\20211831\AppData\Local\Temp\ipykernel_13188\2122438702.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-01-01 00:00:00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  subset.at[index, 'year'] = datetime.strptime(str(value), '%Y')


### Additional cleaning

In [23]:
for column in new_df.columns:
    for index, value in new_df[column].items():
        if str(value).startswith(','):
            new_df.at[index, column] = np.nan
        if str(value).startswith('Surgery'):             # from manual check 
            new_df.at[index, column] = np.nan

In [24]:
nan_rows = new_df.iloc[:, 7:17].isnull().all(axis=1)
new_df = new_df[~nan_rows]
new_df.reset_index(drop=True, inplace=True)


In [25]:
for index, row in new_df.iterrows():
    for i in range(1, 10):
        current_col = 'contributor_' + str(i)
        next_col = 'contributor_' + str(i + 1)
        
        # Check if current column is NaN and next column is not NaN
        if pd.isna(row[current_col]) and not pd.isna(row[next_col]):
            print(row)
            # Replace current column with next column and set next column to NaN
            new_df.at[index, current_col] = row[next_col]
            new_df.at[index, next_col] = np.nan

integer_id                                                       77423
thesis_identifier      https://dspace.library.uu.nl/handle/1874/416508
institution                                                       umcu
author_name                                       Hol, Janna Aldegonda
title                Renal tumors in children and the role of (epi)...
year                                               2022-01-01 00:00:00
language                                                            en
contributor_1                                                      NaN
contributor_2                                             Jongmans, M.
contributor_3                                                      NaN
contributor_4                                                      NaN
contributor_5                                                      NaN
contributor_6                                                      NaN
contributor_7                                                      NaN
contri

In [26]:
df = new_df
df = df.dropna(subset=['author_name', 'title', 'contributor_1'])
df.reset_index(drop=True, inplace=True)

In [27]:
df = df.drop_duplicates(subset=['author_name'], keep='last')
df.reset_index(drop=True, inplace=True)

### Save to CSV - USE IN `Open_Alex_Final.ipynb`

In [28]:
df.to_csv('biomedical_data.csv', index=False)